# Generated by alkh

In [1]:
import sys
sys.path.insert(0, r'/mnt/D/tech/repos/alkh/tests')
sys.path.insert(1, r'/snap/pycharm-community/315/plugins/python-ce/helpers/pydev')
sys.path.insert(2, r'/snap/pycharm-community/315/plugins/python-ce/helpers/third_party/thriftpy')
sys.path.insert(3, r'/snap/pycharm-community/315/plugins/python-ce/helpers/pydev')
sys.path.insert(4, r'/mnt/D/tech/repos/alkh')
sys.path.insert(5, r'/home/erez/.cache/JetBrains/PyCharmCE2022.3/cythonExtensions')
sys.path.insert(6, r'/mnt/D/tech/repos/alkh/tests')
sys.path.insert(7, r'/home/erez/anaconda3/lib/python39.zip')
sys.path.insert(8, r'/home/erez/anaconda3/lib/python3.9')
sys.path.insert(9, r'/home/erez/anaconda3/lib/python3.9/lib-dynload')
sys.path.insert(10, r'/home/erez/anaconda3/lib/python3.9/site-packages')
sys.path.insert(11, r'/home/erez/anaconda3/lib/python3.9/site-packages/IPython/extensions')

In [2]:
import pandas as pd
import alkh

In [3]:
from typing import *
import numpy as np
from alkh.logic_core import CallGraphManager
from alkh.utils import code_range_utils
import itertools
import networkx as nx
import libcst as cst

In [4]:
stack_df = pd.read_pickle('2edecce8b9674d5296cf2403c24cb68c.pkl')
stack_df[["file_path", "function", "lineno", "locals_names"]]

,file_path,function,lineno,locals_names
0,/mnt/D/tech/repos/alkh/alkh/logic_core.py,_calc_dependency_graph,304,"[self, assignment_df, target_id_to_line_number..."
1,/mnt/D/tech/repos/alkh/alkh/logic_core.py,__init__,16,"[self, file_path]"
2,/mnt/D/tech/repos/alkh/tests/logic_core_test.py,<module>,6,"[file_path, line_number]"


In [5]:
alkh.print_context(stack_df.loc[0, 'context'])

/mnt/D/tech/repos/alkh/alkh/logic_core.py
301: 
302:         self._dependency_graph = di_graph
303:         pass
304: 
305:     @staticmethod


In [6]:
self = stack_df.loc[0, 'locals']['self']
assignment_df = stack_df.loc[0, 'locals']['assignment_df']
target_id_to_line_numbers_df = stack_df.loc[0, 'locals']['target_id_to_line_numbers_df']
di_graph = stack_df.loc[0, 'locals']['di_graph']
var_names = stack_df.loc[0, 'locals']['var_names']
name = stack_df.loc[0, 'locals']['name']
index = stack_df.loc[0, 'locals']['index']
row_series = stack_df.loc[0, 'locals']['row_series']

In [7]:
file_path = 'play3.py'

In [8]:
file_lines = open(file_path, 'r').readlines()
file_content = open(file_path, 'r').read()
wrapper = cst.metadata.MetadataWrapper(cst.parse_module(file_content))
scopes = set(wrapper.resolve(cst.metadata.ScopeProvider).values())
ranges = wrapper.resolve(cst.metadata.PositionProvider)
file_number_of_lines = len(file_lines)

In [9]:
class CallCollector(cst.CSTVisitor):
    def __init__(self, ranges):
        super().__init__()
        self._ranges = ranges
        self._calls = []

    def visit_Call(self, node: cst.Assign) -> None:
        print(self._ranges[node])
        print(node.func)
        if isinstance(node.func, cst.Name):
            print('name')
            self._calls.append(((node.func.value,), self._ranges[node]))
        elif isinstance(node.func, cst.Attribute):
            print('att')
            self._calls.append(((node.func.value.value, node.func.attr.value), self._ranges[node]))

In [10]:
visitor = CallCollector(ranges)
a = wrapper.visit(visitor)

CodeRange(start=CodePosition(line=3, column=8), end=CodePosition(line=3, column=16))
Attribute(
    value=Name(
        value='self',
        lpar=[],
        rpar=[],
    ),
    attr=Name(
        value='a',
        lpar=[],
        rpar=[],
    ),
    dot=Dot(
        whitespace_before=SimpleWhitespace(
            value='',
        ),
        whitespace_after=SimpleWhitespace(
            value='',
        ),
    ),
    lpar=[],
    rpar=[],
)
att
CodeRange(start=CodePosition(line=7, column=8), end=CodePosition(line=7, column=16))
Name(
    value='a_func',
    lpar=[],
    rpar=[],
)
name
CodeRange(start=CodePosition(line=12, column=12), end=CodePosition(line=12, column=20))
Attribute(
    value=Name(
        value='self',
        lpar=[],
        rpar=[],
    ),
    attr=Name(
        value='a',
        lpar=[],
        rpar=[],
    ),
    dot=Dot(
        whitespace_before=SimpleWhitespace(
            value='',
        ),
        whitespace_after=SimpleWhitespace(
            val

In [11]:
calls_df = pd.DataFrame(visitor._calls, columns=['target', 'node_range'])
calls_df

,target,node_range
0,"(self, a)","CodeRange(start=CodePosition(line=3, column=8)..."
1,"(a_func,)","CodeRange(start=CodePosition(line=7, column=8)..."
2,"(self, a)","CodeRange(start=CodePosition(line=12, column=1..."
3,"(a_func,)","CodeRange(start=CodePosition(line=16, column=8..."


In [ ]:
assignment_df

In [ ]:
self._scopes_df

In [ ]:
calls_df.iloc[0]

In [ ]:
self._scopes_df

In [ ]:
self._class_scopes_df

In [ ]:
call_series = calls_df.iloc[0]
call_series

In [ ]:
node_range = call_series["node_range"]
node_range

In [ ]:
does_contain_in_class_series = self._class_scopes_df["node_range"].apply(code_range_utils.check_code_range_a_contains_b, args=(node_range,))
does_contain_in_class_series

In [ ]:
containing_class_series = self._class_scopes_df[does_contain_in_class_series].iloc[0]
containing_class_series

In [ ]:
is_function_scope_df = self._scopes_df["scope"].apply(lambda x: isinstance(x, cst.metadata.scope_provider.FunctionScope))
is_function_scope_df

In [ ]:
functions_scopes_df = self._scopes_df[is_function_scope_df]
functions_scopes_df

In [ ]:
call_series["target"][1]

In [ ]:
is_function_within_class = functions_scopes_df["node_range"].apply(code_range_utils.check_code_range_a_is_within_b, args=(containing_class_series["node_range"],))
is_function_within_class

In [ ]:
relevant_functions_scopes_df = functions_scopes_df[is_function_within_class]
relevant_functions_scopes_df

In [ ]:
does_same_name_series = relevant_functions_scopes_df["name"] == call_series["target"][1]
does_same_name_series

In [ ]:
relevant_scope_index = relevant_functions_scopes_df[does_same_name_series].iloc[0]["scope_index"]

In [ ]:
calls_df

In [ ]:
def get_scope_index_for_self_call(call_series):
    node_range = call_series["node_range"]
    does_contain_in_class_series = self._class_scopes_df["node_range"].apply(code_range_utils.check_code_range_a_contains_b, args=(node_range,))
    containing_class_series = self._class_scopes_df[does_contain_in_class_series].iloc[0]
    is_function_scope_df = self._scopes_df["scope"].apply(lambda x: isinstance(x, cst.metadata.scope_provider.FunctionScope))
    functions_scopes_df = self._scopes_df[is_function_scope_df]
    is_function_within_class = functions_scopes_df["node_range"].apply(code_range_utils.check_code_range_a_is_within_b, args=(containing_class_series["node_range"],))
    relevant_functions_scopes_df = functions_scopes_df[is_function_within_class]
    does_same_name_series = relevant_functions_scopes_df["name"] == call_series["target"][1]
    relevant_scope_index = relevant_functions_scopes_df[does_same_name_series].iloc[0]["scope_index"]
    return relevant_scope_index

In [ ]:
def get_scope_index_for_call(call_series):
    if call_series["target"][0] == "self":
        scope_index = get_scope_index_for_self_call(call_series)
    else:
        scope_index = 0
    return scope_index

In [ ]:
calls_df.apply(get_scope_index_for_call, axis=1)

In [ ]:
calls_df

In [ ]:
call_series = calls_df.iloc[1]
call_series

In [ ]:
functions_scopes_df

In [ ]:
function_node_range = functions_scopes_df.iloc[2]["node_range"]
function_node_range

In [ ]:
self._scopes_df["node_range"].apply(code_range_utils.check_code_range_a_contains_b, args=(function_node_range,)).sum() == 2

In [ ]:
def is_function_global(node_range):
    is_global = self._scopes_df["node_range"].apply(code_range_utils.check_code_range_a_contains_b, args=(node_range,)).sum() == 2
    return is_global

In [ ]:
functions_scopes_df["node_range"].apply(is_function_global)

In [ ]:
functions_scopes_df

In [ ]:
is_function_global_series = functions_scopes_df["node_range"].apply(is_function_global)
is_function_global_series

In [ ]:
global_functions_df = functions_scopes_df[is_function_global_series]
global_functions_df

In [ ]:
does_same_name_series = global_functions_df["name"] == call_series["target"][0]
does_same_name_series

In [ ]:
relevant_scope_index = global_functions_df[does_same_name_series].iloc[0]["scope_index"]

In [12]:
def get_scope_index_for_self_call(call_series):
    node_range = call_series["node_range"]
    does_contain_in_class_series = self._class_scopes_df["node_range"].apply(code_range_utils.check_code_range_a_contains_b, args=(node_range,))
    containing_class_series = self._class_scopes_df[does_contain_in_class_series].iloc[0]
    is_function_scope_df = self._scopes_df["scope"].apply(lambda x: isinstance(x, cst.metadata.scope_provider.FunctionScope))
    functions_scopes_df = self._scopes_df[is_function_scope_df]
    is_function_within_class = functions_scopes_df["node_range"].apply(code_range_utils.check_code_range_a_is_within_b, args=(containing_class_series["node_range"],))
    relevant_functions_scopes_df = functions_scopes_df[is_function_within_class]
    does_same_name_series = relevant_functions_scopes_df["name"] == call_series["target"][1]
    relevant_scope_index = relevant_functions_scopes_df[does_same_name_series].iloc[0]["scope_index"]
    return relevant_scope_index

def is_function_global(node_range):
    is_global = self._scopes_df["node_range"].apply(code_range_utils.check_code_range_a_contains_b, args=(node_range,)).sum() == 2
    return is_global

def get_scope_index_for_global_call(call_series):
    node_range = call_series["node_range"]
    is_function_scope_df = self._scopes_df["scope"].apply(lambda x: isinstance(x, cst.metadata.scope_provider.FunctionScope))
    functions_scopes_df = self._scopes_df[is_function_scope_df]
    is_function_global_series = functions_scopes_df["node_range"].apply(is_function_global)
    global_functions_df = functions_scopes_df[is_function_global_series]
    does_same_name_series = global_functions_df["name"] == call_series["target"][0]
    relevant_scope_index = global_functions_df[does_same_name_series].iloc[0]["scope_index"]
    return relevant_scope_index

In [13]:
def get_scope_index_for_call(call_series):
    if call_series["target"][0] == "self":
        scope_index = get_scope_index_for_self_call(call_series)
    else:
        scope_index = get_scope_index_for_global_call(call_series)
    return scope_index

In [14]:
calls_df["scope_index"] = calls_df.apply(get_scope_index_for_call, axis=1)

In [15]:
calls_df

,target,node_range,scope_index
0,"(self, a)","CodeRange(start=CodePosition(line=3, column=8)...",5
1,"(a_func,)","CodeRange(start=CodePosition(line=7, column=8)...",2
2,"(self, a)","CodeRange(start=CodePosition(line=12, column=1...",7
3,"(a_func,)","CodeRange(start=CodePosition(line=16, column=8...",2
